# Date as an input Argument

In [1]:
import boto3
import pandas as pd
from io import StringIO, BytesIO
from datetime import datetime, timedelta

In [2]:
arg_date = "2021-05-09"
src_format = "%Y-%m-%d"
src_bucket = "deutsche-boerse-xetra-pds"
trg_bucket = "xetra--1234"
columns = ['ISIN', 'Date', 'Time', 'StartPrice', 'MaxPrice', 'MinPrice', 'EndPrice', 'TradedVolume']
key = "xetra_daily_report" + datetime.today().strftime("%Y%m%d_%H%M%S") + ".parquet"

In [3]:
arg_date_dt = datetime.strptime(arg_date, src_format).date() - timedelta(days = 1)

In [ ]:
# arg_date_dt

In [4]:
arg_date_end = datetime.strptime(arg_date, src_format).date() + timedelta(days = 2)

In [ ]:
# arg_date_end

In [5]:
s3 = boto3.resource("s3")
bucket = s3.Bucket(src_bucket)
# objects = [obj for obj in bucket.objects.all() if datetime.strptime(obj.key.split("/")[0], '%Y-%m-%d').date() >= arg_date_dt]
objects = [obj for obj in bucket.objects.all() if datetime.strptime(obj.key.split("/")[0], src_format).date() >= arg_date_dt 
           if datetime.strptime(obj.key.split("/")[0], src_format).date() <= arg_date_end]

In [ ]:
#objects = []
#for obj in bucket.objects.all():
   # d = datetime.strptime(obj.key.split("/")[0], '%Y-%m-%d').date()
   # if d >= arg_date_dt and d <= arg_date_end:
     #   objects.append(obj)

In [ ]:
# objects

In [ ]:
# df_init.columns

In [6]:
def csv_to_df(filename):
    csv_obj = bucket.Object(key = filename).get().get("Body").read().decode("utf-8")
    data = StringIO(csv_obj)
    df = pd.read_csv(data, delimiter = ",")
    return df
df_all = pd.concat([csv_to_df(obj.key) for obj in objects], ignore_index = True)

In [8]:
# columns = ['ISIN', 'Date', 'Time', 'StartPrice', 'MaxPrice', 'MinPrice', 'EndPrice', 'TradedVolume']
df_all = df_all.loc[:, columns]

In [9]:
df_all.dropna(inplace = True)

## Get opening price per ISIN and Day

In [10]:
df_all["opening_price"] = df_all.sort_values(by=["Time"]).groupby(["ISIN", "Date"])["StartPrice"].transform("first")

## Get closing price per ISIN and Day

In [11]:
df_all["closing_price"] = df_all.sort_values(by=["Time"]).groupby(["ISIN", "Date"])["StartPrice"].transform("last")

## Aggregation

In [12]:
df_all = df_all.groupby(["ISIN", "Date"], as_index = False).agg(opening_price_eur = ("opening_price", "min"),
                                                                closing_price_eur = ("closing_price", "min"), 
                                                                minimum_price_eur = ("MinPrice", "min"),
                                                                maximum_price_eur = ("MaxPrice", "max"),
                                                                daily_traded_volume = ("TradedVolume", "sum"))

## Percent Change Prev Closing

In [13]:
df_all["prev_closing_price"] = df_all.sort_values(by = ["Date"]).groupby(["ISIN"])["closing_price_eur"].shift(1)

In [14]:
df_all["change_prev_closing_%"] = (df_all["closing_price_eur"] - df_all["prev_closing_price"]) / df_all["prev_closing_price"] * 100

In [15]:
df_all.drop(columns = ["prev_closing_price"], inplace = True)

In [16]:
df_all = df_all.round(2)

In [17]:
df_all = df_all[df_all.Date >= arg_date]

## Write to S3

In [18]:
out_buffer = BytesIO()
df_all.to_parquet(out_buffer, index = False)
bucket_target = s3.Bucket(trg_bucket)
bucket_target.put_object(Body = out_buffer.getvalue(), Key = key)

s3.Object(bucket_name='xetra--1234', key='xetra_daily_report20220209_222954.parquet')

## Read the uploaded file

In [19]:
for obj in bucket_target.objects.all():
    print(obj.key)

xetra_daily_report20220209_205648.parquet
xetra_daily_report20220209_222954.parquet


In [20]:
prq_obj = bucket_target.Object(key = "xetra_daily_report20220209_222954.parquet").get().get("Body").read()
data = BytesIO(prq_obj)
df_report = pd.read_parquet(data)

In [21]:
df_report

ISIN        Date  opening_price_eur  closing_price_eur  \
0     AT00000FACC2  2021-05-10               9.10               9.02   
1     AT00000FACC2  2021-05-11               8.85               8.82   
2     AT0000606306  2021-05-10              18.75              19.27   
3     AT0000606306  2021-05-11              19.11              19.12   
4     AT0000609607  2021-05-10              16.48              17.46   
...            ...         ...                ...                ...   
5911  XS2265369731  2021-05-11              10.28              10.16   
5912  XS2265370234  2021-05-10              24.23              24.59   
5913  XS2265370234  2021-05-11              24.42              24.09   
5914  XS2284324667  2021-05-10              26.87              26.63   
5915  XS2284324667  2021-05-11              26.58              26.80   

      minimum_price_eur  maximum_price_eur  daily_traded_volume  \
0                  8.94               9.10                  145   
1                  8.82               8.85                  304   
2                 18.68              19.30                 3077   
3                 19.01              19.16                 3447   
4                 16.48              17.46                  331   
...                 ...                ...                  ...   
5911              10.16              10.30                    0   
5912              24.23              24.59                  300   
5913              24.07              24.49                  600   
5914              26.57              26.90                 8302   
5915              26.47              26.80                 2633   

      change_prev_closing_%  
0                       NaN  
1                     -2.22  
2                       NaN  
3                     -0.78  
4                       NaN  
...                     ...  
5911                  -2.73  
5912                    NaN  
5913                  -2.03  
5914                    NaN  
5915                   0.65  

[5916 rows x 8 columns]